In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('house-votes-84.data', header=None, names=['class', 'h-infants', 'wproject-cost-sharing', 'adoption-of-the-budget-resolution', 'physician-fee-freeze', 'el-salvador-aid', 'religious-in-schools', 'anti-satellite', 'aid-to-nicaraguan', 'mx-missile', 'immigration', 'synfuels-cutback', 'education-spending', 'superfund-right-to-sue', 'crime', 'duty-free', 'export-south-africa'])
df.head(3)

In [ ]:
df.info()

In [ ]:
def draw_histograms(dataframe, features, rows, cols):
    fig=plt.figure(figsize=(20,10))
    for i, feature in enumerate(features):
        ax=fig.add_subplot(rows,cols,i+1)
        dataframe[feature].hist(bins=20,ax=ax,facecolor='midnightblue')
        ax.set_title(feature+" Distribution",color='DarkRed')
        
    fig.tight_layout()  
    plt.show()
draw_histograms(df,df.columns,6,3)

In [ ]:
# Retirar as colunas contendo '?', perderia muitos dados. Vamos usar a base toda, pois o xgboosting aceita nan

new = df[df != '?']
new.dropna()

In [ ]:
# Substituindo n = o e y =1 e ? = NaN
df[df == 'n'] = 0
df[df == 'y'] = 1
df = df[df != '?']

In [ ]:
df

In [ ]:
X = df.iloc[:,1:].values
y = df.iloc[:,:1].values

In [ ]:
# dividindo em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, random_state = 7)

In [ ]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [value for value in y_pred]

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))